In [ ]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas
import plotly.express as px
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AnimalShelterRepository import AnimalShelterRepository


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"
repo = AnimalShelterRepository(username, password)

# class read method must support return of cursor object and accept projection json input
query = repo.query({ }, 1, 10000)
for record in query:
    del record['_id'] # it doesn't like the ObjectId object.
df = pandas.DataFrame.from_records(query)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Header("Mark Holden SNHU CS-340 MongoDB GeoLocation"),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6'),
    html.Br(),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="multi",
        sort_action='native',
        filter_action='native'
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

# No it doesn't. It just causes errors and doesn't give you anything.

# @app.callback(
#     Output('datatable-id', 'style_data_conditional'),
#     [Input('datatable-id', 'selected_columns')]
# )
# def update_styles(selected_columns):
#     return [{
#         'if': { 'column_id': i },
#         'background_color': '#D2F3FF'
#     } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])
def update_map(viewData, selected):
    if (selected is None):
        selected = [0]
        center = [30.75,-97.48] # Austin TX is at [30.75,-97.48]

    dff = pandas.DataFrame.from_dict(viewData)

    children = [ dl.TileLayer(id="base-layer-id") ]
    if (dff is not None):
        # Marker with tool tip and popup
        marker = [dl.Marker(position=[dff.iloc[i,13], dff.iloc[i,14]], children=[
                dl.Tooltip(dff.iloc[i,4]),
                dl.Popup([ html.H1(dff.iloc[i,9]) ])
            ]) for i in selected]
        for i in range(len(marker)):
            children.append(marker[i])

    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=center, zoom=10, children=children) ]

app.run_server(mode='external')
